In [ ]:
# Utilities
import numpy as np
import pathlib
import PIL 

# ML libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Access dataset using paths
frames = pathlib.Path("./frames")

train_frames = frames / "train"
train_norm_frames = train_frames / "norm"
train_weap_frames = train_frames / "weap"

test_frames = frames / "test"
test_norm_frames = test_frames / "norm"
test_weap_frames = test_frames / "weap"

In [ ]:
# Basic dataset statistics
print("Number of training frames: {}".format(len(list(train_frames.glob("**/*")))))
print("Training frames statistics:")
print("\tNumber of normal frames: {}".format(len(list(train_norm_frames.glob("**/*")))))
print("\tNumber of weapon frames: {}".format(len(list(train_weap_frames.glob("**/*")))))
print()
print("Number of testing frames: {}".format(len(list(test_frames.glob("**/*")))))
print("Testing frames statistics:")
print("\tNumber of normal frames: {}".format(len(list(test_norm_frames.glob("**/*")))))
print("\tNumber of weapon frames: {}".format(len(list(test_weap_frames.glob("**/*")))))

In [ ]:
# Load dataset
batch_size = 32

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    train_frames,
    validation_split=0.2,
    subset="both",
    seed=123,
    batch_size=batch_size
)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
# Dataset configuration
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Dataset standardization
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
# Basic Keras model
num_classes = len(class_names)
model = Sequential([
  layers.Rescaling(1./255, input_shape=(256, 256, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

# Compile model
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
# Train the model
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
# Visualize training results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = np.array(range(epochs))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Test the model
num_frames = 0
num_norm_frames = 0
num_weap_frames = 0
num_norm_correct = 0
num_weap_correct = 0

for frame in test_norm_frames.glob("**/*.png"):
    num_frames += 1
    num_norm_frames += 1
    img = tf.keras.utils.load_img(
        frame, target_size=(256, 256)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array, verbose=0)
    score = tf.nn.softmax(predictions[0])
    if class_names[np.argmax(score)] == "norm":
        num_norm_correct += 1

for frame in test_weap_frames.glob("**/*.png"):
    num_frames += 1
    num_weap_frames += 1
    img = tf.keras.utils.load_img(
        frame, target_size=(256, 256)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model.predict(img_array, verbose=0)
    score = tf.nn.softmax(predictions[0])
    if class_names[np.argmax(score)] == "weap":
        num_weap_correct += 1

In [ ]:
# Evaluation
num_correct = num_norm_correct + num_weap_correct
print("The model accurately identified ", num_correct, " out of ", num_frames, " frames")
print("Accuracy = ", num_correct / num_frames)

In [ ]:
# Confusion matrix
confusion_matrix = np.array([[num_weap_correct, num_norm_frames - num_norm_correct], [num_weap_frames - num_weap_correct, num_norm_correct]])
labels = ["weap", "norm"]

plt.figure(figsize=(6, 4))
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)

plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')